# Exploration of the data

The notebook can not be run without having a `data` folder with the data from the ``https://round-dataset.com`` website.


The functions to format the data are adapted from `https://github.com/RobertKrajewski/highD-dataset/blob/master/Python/src/data_management/read_csv.py`. And the ones to plot the data from `https://github.com/ika-rwth-aachen/drone-dataset-tools/blob/master/src/track_visualizer.py`.

In [1]:
# Load packages
import imageio
import glob
import numpy as np
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

COLORS = ["#377eb8", "#ff7f00", "#4daf4a",
          "#f781bf", "#a65628", "#984ea3",
          "#999999", "#e41a1c", "#dede00"]
custom_palette = sns.set_palette(sns.color_palette(COLORS))

In [2]:
# TRACK FILE
RECORDING_ID = "recordingId"  # -
TRACK_ID = "trackId"  # -
FRAME = "frame"
TRACK_LIFETIME = "trackLifetime"  # -
BBOX = "bbox"  # (m, m, m, m)
X_CENTER = "xCenter"  # m
Y_CENTER = "yCenter"  # m
HEADING = "heading"  # deg
WIDTH = "width"  # m
HEIGHT = "height"  # m
X_VELOCITY = "xVelocity"  # m/s
Y_VELOCITY = "yVelocity"  # m/s
X_ACCELERATION = "xAcceleration"  # m/s^2
Y_ACCELERATION = "yAcceleration"  # m/s^2
LON_VELOCITY = "lonVelocity"  # m/s
LAT_VELOCITY = "latVelocity"  # m/s
LON_ACCELERATION = "lonAcceleration"  # m/s^2
LAT_ACCELERATION = "latAcceleration"  # m/s^2

# STATIC FILE
RECORDING_ID = "recordingId"  # -
TRACK_ID = "trackId"  # -
INITIAL_FRAME = "initialFrame"  # -
FINAL_FRAME = "finalFrame"  # -
NUM_FRAMES = "numFrames"  # -
WIDTH = "width"  # m
LENGTH = "length"  # m
CLASS = "class"  # -

# VIDEO META
RECORDING_ID = "recordingId"  # -
LOCATION_ID = "locationId"  # -
FRAME_RATE = "frameRate"  # hz
SPEED_LIMIT = "speedLimit"  # m/s
WEEKDAY = "weekday"  # -
START_TIME = "startTime"  # hh
DURATION = "duration"  # s
N_TRACKS = "numTracks"  # -
N_VEHICLES = "numVehicles"  # -
N_VRUS = 'numVRUs'  # -
LAT_LOCATION = 'latLocation'  # deg
LON_LOCATION = 'lonLocation'  # deg
X_UTM_ORIGIN = 'xUtmOrigin'  # m
Y_UTM_ORIGIN = 'yUtmOrigin'  # m
ORTHO_PX_TO_METER = 'orthoPxToMeter'  # m/px

In [3]:
# Function to read {}_recordingMeta.csv
def read_meta_info(arguments):
    """
    This method reads the video meta file from rounD data.
    :param arguments: the parsed arguments for the program containing the input path for the video meta csv file.
    :return: the meta dictionary containing the general information of the video
    """
    # Read the csv file, convert it into a useful data structure
    df = pd.read_csv(arguments["input_meta_path"])

    # Declare and initialize the extracted_meta_dictionary
    extracted_meta_dictionary = {RECORDING_ID: int(df[RECORDING_ID][0]),
                                 LOCATION_ID: int(df[LOCATION_ID][0]),
                                 FRAME_RATE: int(df[FRAME_RATE][0]),
                                 SPEED_LIMIT: float(df[SPEED_LIMIT][0]),
                                 WEEKDAY: str(df[WEEKDAY][0]),
                                 START_TIME: str(df[START_TIME][0]),
                                 DURATION: float(df[DURATION][0]),
                                 N_TRACKS: int(df[N_TRACKS][0]),
                                 N_VEHICLES: int(df[N_VEHICLES][0]),
                                 N_VRUS: int(df[N_VRUS][0]),
                                 LAT_LOCATION: float(df[LAT_LOCATION][0]),
                                 LON_LOCATION: float(df[LON_LOCATION][0]),
                                 X_UTM_ORIGIN: float(df[X_UTM_ORIGIN][0]),
                                 Y_UTM_ORIGIN: float(df[Y_UTM_ORIGIN][0]),
                                 ORTHO_PX_TO_METER: float(df[ORTHO_PX_TO_METER][0])}
    return extracted_meta_dictionary

In [4]:
# Function to read {}_tracksMeta.csv
def read_static_info(arguments):
    """
    This method reads the static info file from rounD data.
    :param arguments: the parsed arguments for the program containing the input path for the static csv file.
    :return: the static dictionary - the key is the track_id and the value is the corresponding data for this track
    """
    # Read the csv file, convert it into a useful data structure
    df = pd.read_csv(arguments["input_static_path"])

    # Declare and initialize the static_dictionary
    static_dictionary = {}

    # Iterate over all rows of the csv because we need to create the bounding boxes for each row
    for i_row in range(df.shape[0]):
        track_id = int(df[TRACK_ID][i_row])
        static_dictionary[track_id] = {TRACK_ID: track_id,
                                       INITIAL_FRAME: int(df[INITIAL_FRAME][i_row]),
                                       FINAL_FRAME: int(df[FINAL_FRAME][i_row]),
                                       NUM_FRAMES: int(df[NUM_FRAMES][i_row]),
                                       WIDTH: float(df[WIDTH][i_row]),
                                       LENGTH: float(df[LENGTH][i_row]),
                                       CLASS: str(df[CLASS][i_row])}
    return static_dictionary

In [5]:
# Function to read {}_track.csv
def read_track_csv(arguments):
    """
    This method reads the tracks file from rounD data.
    :param arguments: the parsed arguments for the program containing the input path for the tracks csv file.
    :return: a list containing all tracks as dictionaries.
    """
    # Get some metainformation on the recording
    meta = read_meta_info(arguments)
    
    # Read the csv file, convert it into a useful data structure
    df = pd.read_csv(arguments["input_path"])

    # Use groupby to aggregate track info. Less error prone than iterating over the data.
    grouped = df.groupby([TRACK_ID], sort=False)
    # Efficiently pre-allocate an empty list of sufficient size
    tracks = [None] * grouped.ngroups
    current_track = 0
    for group_id, rows in grouped:
        bounding_boxes = np.transpose(np.array([rows[X_CENTER].values,
                                                rows[Y_CENTER].values,
                                                rows[WIDTH].values,
                                                rows[LENGTH].values]))
        tracks[current_track] = {RECORDING_ID: meta.get('recordingId'),
                                 TRACK_ID: np.int64(group_id),  # for compatibility, int would be more space efficient
                                 FRAME: rows[FRAME].values,
                                 TRACK_LIFETIME: rows[TRACK_LIFETIME].values,
                                 BBOX: bounding_boxes,
                                 X_CENTER: rows[X_CENTER].values,
                                 Y_CENTER: rows[Y_CENTER].values,
                                 HEADING: rows[HEADING].values,
                                 WIDTH: rows[WIDTH].values,
                                 LENGTH: rows[LENGTH].values,
                                 X_VELOCITY: rows[X_VELOCITY].values,
                                 Y_VELOCITY: rows[Y_VELOCITY].values,
                                 X_ACCELERATION: rows[X_ACCELERATION].values,
                                 Y_ACCELERATION: rows[Y_ACCELERATION].values,
                                 LON_VELOCITY: rows[LON_VELOCITY].values,
                                 LAT_VELOCITY: rows[LAT_VELOCITY].values,
                                 LON_ACCELERATION: rows[LON_ACCELERATION].values,
                                 LAT_ACCELERATION: rows[LAT_ACCELERATION].values
                                }
        current_track = current_track + 1
    return tracks

In [6]:
# Function to read {}_background.png
def read_background(arguments):
    """
    This method reads the background file from rounD data.
    :param arguments: the parsed arguments for the program containing the input path for the background file.
    :return: the background image.
    """
    return imageio.imread(arguments["input_img"])

## Load one file and explore it

In [7]:
RECORDING_NUMBER = '00'

argv = {
    "input_path": f'./data/{RECORDING_NUMBER}_tracks.csv',
    "input_static_path": f'./data/{RECORDING_NUMBER}_tracksMeta.csv',
    "input_meta_path": f'./data/{RECORDING_NUMBER}_recordingMeta.csv',
    "input_img": f'./data/{RECORDING_NUMBER}_background.png'
}

In [8]:
tracks = read_track_csv(argv)
meta_tracks = read_meta_info(argv)
info_tracks = read_static_info(argv)
background = read_background(argv)

In [9]:
def get_id(tracks, idx):
    """
    This method returns the information about an particular id.
    :param tracks: a list contaning all tracks as dictionary
    :param idx: the index to return
    :return: a dictionary containing information for a particular idx"""
    l = [d for d in tracks if d[TRACK_ID] == idx]
    return l[0]

In [10]:
def norm(*args):
    """
    Compute the norm of vectors
    :param args: vectors
    :return: norm
    """
    return np.sqrt(np.sum([i**2 for i in args], axis=0))

In [12]:
def plot_tracks(tracks, idx):
    """
    This methods plots a particular track given an idx.
    :param tracks: a list containing all tracks as dictionary
    :param idx: the indices to plot
    :return: matplotlib figure
    """
    fig, axs = plt.subplots(1, 6, figsize=(15, 5))
    
    for i in idx:
        track = get_id(tracks, i)
        argvals = np.linspace(0, 1, np.size(track[X_VELOCITY]))
        
        axs[0].plot(argvals, track[X_CENTER])
        axs[1].plot(argvals, track[Y_CENTER])
        axs[2].plot(argvals, track[X_VELOCITY])
        axs[3].plot(argvals, track[Y_VELOCITY])
        axs[4].plot(argvals, track[X_ACCELERATION])
        axs[5].plot(argvals, track[Y_ACCELERATION])
    
    axs[0].set_title('Longitudinal Position')
    axs[0].set_xlabel('Normalized time')
    axs[0].set_ylabel(r'X ($m$)')
    
    axs[1].set_title('Lateral Position')
    axs[1].set_xlabel('Normalized time')
    axs[1].set_ylabel(r'Y ($m$)')
    
    axs[2].set_title('Longitudinal Velocity')
    axs[2].set_xlabel('Normalized time')
    axs[2].set_ylabel(r'Velocity ($m/s$)')
    
    axs[3].set_title('Lateral Velocity')
    axs[3].set_xlabel('Normalized time')
    axs[3].set_ylabel(r'Velocity ($m/s$)')
    
    axs[4].set_title('Longitudinal Acceleration')
    axs[4].set_xlabel('Normalized time')
    axs[4].set_ylabel(r'Acceleration ($m/s^2$)')
    
    axs[5].set_title('Lateral Acceleration')
    axs[5].set_xlabel('Normalized time')
    axs[5].set_ylabel(r'Acceleration ($m/s^2$)')
    
    fig.tight_layout()
    return fig, axs

In [13]:
idx = np.random.randint(1, len(tracks), 5)

In [14]:
plot_tracks(tracks, idx)
plt.savefig(f'./figures/recording_{RECORDING_NUMBER}/sample.pdf')

## Visualization using the background image

In [15]:
# Utility functions
def cart2pol(cart):
    """
    Transform cartesian to polar coordinates.
    :param cart: Nx2 ndarray
    :return: 2 Nx1 ndarrays
    """
    if cart.shape == (2,):
        cart = np.array([cart])

    x = cart[:, 0]
    y = cart[:, 1]

    th = np.arctan2(y, x)
    r = np.sqrt(np.power(x, 2) + np.power(y, 2))
    return th, r

def pol2cart(th, r):
    """
    Transform polar to cartesian coordinates.
    :param th: Nx1 ndarray
    :param r: Nx1 ndarray
    :return: Nx2 ndarray
    """

    x = np.multiply(r, np.cos(th))
    y = np.multiply(r, np.sin(th))

    cart = np.array([x, y]).transpose()
    return cart

def calculate_rotated_bboxes(center_points_x, center_points_y, length, width, rotation=0):
    """
    Calculate bounding box vertices from centroid, width and length.
    :param centroid: center point of bbox
    :param length: length of bbox
    :param width: width of bbox
    :param rotation: rotation of main bbox axis (along length)
    :return:
    """

    centroid = np.array([center_points_x, center_points_y]).transpose()

    centroid = np.array(centroid)
    if centroid.shape == (2,):
        centroid = np.array([centroid])

    # Preallocate
    data_length = centroid.shape[0]
    rotated_bbox_vertices = np.empty((data_length, 4, 2))

    # Calculate rotated bounding box vertices
    rotated_bbox_vertices[:, 0, 0] = -length / 2
    rotated_bbox_vertices[:, 0, 1] = -width / 2

    rotated_bbox_vertices[:, 1, 0] = length / 2
    rotated_bbox_vertices[:, 1, 1] = -width / 2

    rotated_bbox_vertices[:, 2, 0] = length / 2
    rotated_bbox_vertices[:, 2, 1] = width / 2

    rotated_bbox_vertices[:, 3, 0] = -length / 2
    rotated_bbox_vertices[:, 3, 1] = width / 2

    for i in range(4):
        th, r = cart2pol(rotated_bbox_vertices[:, i, :])
        rotated_bbox_vertices[:, i, :] = pol2cart(th + rotation, r).squeeze()
        rotated_bbox_vertices[:, i, :] = rotated_bbox_vertices[:, i, :] + centroid

    return rotated_bbox_vertices

In [19]:
def plot_trajectories_vel(tracks, idx, img, meta_tracks, sdf=10, skip_frame=10):
    """
    This methods plots a particular trajectory given an idx.
    :param tracks: a list containing all tracks as dictionary
    :param idx: the indices to plot
    :param img: the background image
    :param sdf: scale down factor (default=10)
    :param skip_frame: number of frames to skip for plotting (defualt=10)
    :return: matplotlib figure
    """
    # Set parameters
    ortho_px_to_meter = meta_tracks[ORTHO_PX_TO_METER]
    scale_down_factor = sdf
    
    # Define the figure
    fig, axs = plt.subplots(1, 1)
    fig.set_size_inches(18, 8)
    
    j = 0
    for i in idx:
        track = get_id(tracks, i)

        # Define a dictionary for plotting
        track_vis = {}
        track_vis["xCenterVis"] = track[X_CENTER][::skip_frame] / ortho_px_to_meter / scale_down_factor
        track_vis["yCenterVis"] = -track[Y_CENTER][::skip_frame] / ortho_px_to_meter / scale_down_factor
    
        p = axs.plot(track_vis["xCenterVis"],
                     track_vis["yCenterVis"],
                     color=COLORS[j], linewidth=5, alpha=0.75)
        for x, y, vx, vy in zip(track_vis["xCenterVis"], track_vis["yCenterVis"],
                                track[X_VELOCITY][::skip_frame], -track[Y_VELOCITY][::skip_frame]):
            p = axs.arrow(x, y, 10*vx, 10*vy, color=COLORS[j], head_width=10, head_length=10)
        j = j + 1
        
    axs.imshow(img)
    fig.tight_layout()
    return fig, axs

def plot_trajectories_acc(tracks, idx, img, meta_tracks, sdf=10, skip_frame=10):
    """
    This methods plots a particular trajectory given an idx.
    :param tracks: a list containing all tracks as dictionary
    :param idx: the indices to plot
    :param img: the background image
    :param sdf: scale down factor (default=10)
    :param skip_frame: number of frames to skip for plotting (defualt=10)
    :return: matplotlib figure
    """
    # Set parameters
    ortho_px_to_meter = meta_tracks[ORTHO_PX_TO_METER]
    scale_down_factor = sdf
    
    # Define the figure
    fig, axs = plt.subplots(1, 1)
    fig.set_size_inches(18, 8)
    
    j = 0
    for i in idx:
        track = get_id(tracks, i)

        # Define a dictionary for plotting
        track_vis = {}
        track_vis["xCenterVis"] = track[X_CENTER][::skip_frame] / ortho_px_to_meter / scale_down_factor
        track_vis["yCenterVis"] = -track[Y_CENTER][::skip_frame] / ortho_px_to_meter / scale_down_factor
    
        q = axs.plot(track_vis["xCenterVis"],
                     track_vis["yCenterVis"],
                     color=COLORS[j], linewidth=5, alpha=0.75)
        for x, y, ax, ay in zip(track_vis["xCenterVis"], track_vis["yCenterVis"],
                                track[X_ACCELERATION][::skip_frame], -track[Y_ACCELERATION][::skip_frame]):
            q = axs.arrow(x, y, 10*ax, 10*ay, color=COLORS[j], head_width=10, head_length=10)
        j = j + 1

    axs.imshow(img)
    fig.tight_layout()
    return fig, axs

In [21]:
_ = plot_trajectories_vel(tracks, idx, background, meta_tracks, sdf=10, skip_frame=15)
plt.savefig(f'./figures/recording_{RECORDING_NUMBER}/sample_roundabout_vel.pdf')

In [22]:
_ = plot_trajectories_acc(tracks, idx, background, meta_tracks, sdf=10, skip_frame=15)
plt.savefig(f'./figures/recording_{RECORDING_NUMBER}/sample_roundabout_acc.pdf')

## Save only X, Y, xVelocity, yVelocity, xAcceleration and yAcceleration into a pickle file

In [17]:
# Save track as .pkl
pd.to_pickle(tracks, f"./data/processed/recording_{RECORDING_NUMBER}/tracks.pkl")
pd.to_pickle(meta_tracks, f"./data/processed/recording_{RECORDING_NUMBER}/meta.pkl")
pd.to_pickle(info_tracks, f"./data/processed/recording_{RECORDING_NUMBER}/info.pkl")